In [1]:
import pandas as pd
import nltk


In [2]:
file = pd.read_csv('dataset/text_and_label.csv')

In [3]:
data_and_labels = pd.concat([file['title'], file['label']], axis=1)

In [4]:
import numpy as np

msk = np.random.rand(len(data_and_labels)) < 0.8
train = data_and_labels[msk].to_records(index=False)
for i in range(0,len(train)):
    data = {}
    title = train[i][0].split()
    for j in range(0,len(title)):
        data[j] = title[j]
    train[i] = (data, train[i][1])
test = data_and_labels[~msk].to_records(index=False)
for i in range(0,len(test)):
    data = {}
    title = test[i][0].split()
    for j in range(0,len(title)):
        data[j] = title[j]
    test[i] = (data, test[i][1])
print(train)


[({0: "It's", 1: 'not', 2: 'about', 3: 'the', 4: 'money,', 5: "it's", 6: 'about', 7: 'sending', 8: 'a', 9: 'message.', 10: '🚀💎🙌'}, 'Neutral')
 ({0: 'Exit', 1: 'the', 2: 'system'}, 'Neutral')
 ({0: 'NEW', 1: 'SEC', 2: 'FILING', 3: 'FOR', 4: 'GME!', 5: 'CAN', 6: 'SOMEONE', 7: 'LESS', 8: 'RETARDED', 9: 'THAN', 10: 'ME', 11: 'PLEASE', 12: 'INTERPRET?'}, 'Negative')
 ...
 ({0: 'Hitler', 1: 'reacts', 2: 'to', 3: 'the', 4: 'market', 5: 'being', 6: 'irrational'}, 'Negative')
 ({0: 'Daily', 1: 'Discussion', 2: 'Thread', 3: 'for', 4: 'August', 5: '02,', 6: '2021'}, 'Neutral')
 ({0: 'Fraternal', 1: 'Association', 2: 'of', 3: 'Gambling', 4: 'Gentlemen', 5: 'and', 6: 'Yacht', 7: 'Degenerates', 8: 'for', 9: 'August', 10: '02,', 11: '2021'}, 'Neutral')]


In [5]:
from ipynb.fs.full.preprocessing import *
dict_emojis, dict_titles, dict_body = preprocess('dataset/reddit_wsb.csv')
#print(dict_emojis, dict_titles, dict_body)

[preprocess::extract emojis]
[preprocess::extract titles and body]
[preprocess::complete]
[preprocess::extract emojis]
[preprocess::extract titles and body]
[preprocess::complete]


In [6]:
from nltk.corpus import stopwords

def remove_stop_words(dict_sentences):
    stop_words = set(stopwords.words('english'))
    for i in dict_sentences.keys():
        filtered = []
        for word in dict_sentences[i]:
            if word not in stop_words:
                filtered.append(word)
        dict_sentences[i] = filtered
    return dict_sentences

In [7]:
filtered_titles = remove_stop_words(dict_titles)

In [8]:
dataset = []
for i in filtered_titles.keys():
    sentence = filtered_titles[i]
    dict_sentence = {}
    for j in range(0,len(sentence)):
        dict_sentence[j] = sentence[j]
    dataset.append( (dict_sentence, data_and_labels['label'][i] ) )

In [15]:
import random
random.shuffle(dataset)
train = dataset[:int(len(dataset)*0.9)]
test = dataset[int(len(dataset)*0.9):]

In [16]:
classifier = nltk.classify.DecisionTreeClassifier.train(train, entropy_cutoff=0, support_cutoff=0)
sorted(classifier.labels())
#print(classifier)
print(nltk.classify.accuracy(classifier, test))

0.412287793047696
